In [46]:
import geopandas as gpd
import pandas as pd
import os.path
from typing import List, Optional, Dict

In [22]:
hab = gpd.read_file("/mnt/c/Users/MarkBoer/OneDrive - Spheer AI/General/Projecten/Veg_2_Hab/Data/Habitatkarteringen/Habitattypekaarten Gr/NaamGebied_Ruiten Aa.gpkg")

def func(row, habtype_colnames: List[str], percentage_colnames: List[str]=None):
    if percentage_colnames is not None and len(habtype_colnames) != len(percentage_colnames):
        raise ValueError("The number of habitat types and percentages must be equal")
    habtypes = [h for h in row[habtype_colnames] if pd.notnull(h)]
    # TODO valideren dat alle habtypes anders zijn.
    if len(habtypes) == 0:
        return {"H0000": 100}
    if percentage_colnames is None:
        percentages = [100 / len(habtypes)]*len(habtypes)
    else:
        percentages = row[percentage_colnames].tolist()
    # TODO valideren dat percentages optellen tot 100
    return dict(zip(habtypes, percentages))

def parse_habitat_type_percentage(gdf, habtype_cols: List[str], percentage_cols: Optional[List[str]]):
    return gdf.apply(lambda row: func(row, habtype_cols, percentage_cols), axis=1)

ser = parse_habitat_type_percentage(hab, ["Habtype1", "Habtype2", "Habtype3"], None)
ser

0      {'H0000': 100}
1      {'H0000': 100}
2      {'H0000': 100}
3      {'H0000': 100}
4      {'H0000': 100}
            ...      
489    {'H0000': 100}
490    {'H0000': 100}
491    {'H0000': 100}
492    {'H0000': 100}
493    {'H0000': 100}
Length: 494, dtype: object

In [24]:
hab.Habitattyp.value_counts()

+H0000    38
H9120     26
H6410      7
Name: Habitattyp, dtype: int64

In [52]:
def bereken_percentage_correct(habs_pred: Dict[str, float], habs_true: Dict[str, float]):
    pass

# TODO, misschien willen we deze functie een dataframe laten teruggeven met 3 kolommen:
# - pred_hab
# - true_hab
# - percentage
# dan kun je ze daarna makkelijk onder elkaar zetten
def bereken_percentage_confusion_matrix(habs_pred: Dict[str, float], habs_true: Dict[str, float]):
    """ huilie huilie """
    outputs = []
    for pred_hab, pred_percentage in habs_pred.items():
        if pred_hab in habs_true:
            percentage_correct = min(pred_percentage, habs_true[pred_hab])
            outputs.append({
                "pred_hab": pred_hab,
                "true_hab": pred_hab,
                "percentage": percentage_correct,
            })
            habs_pred[pred_hab] -= percentage_correct
            habs_true[pred_hab] -= percentage_correct
    # alle matchende zitten nu in de outputes

    # we houden de volgorde aan van onze prediction
    for pred_hab, pred_percentage in habs_pred.items():
        habs_true = {k: v for k, v in habs_true.items() if v > 0}
        for true_hab, true_percentage in habs_true.items():
            percentage = min(pred_percentage, true_percentage)
            outputs.append({
                "pred_hab": pred_hab,
                "true_hab": true_hab,
                "percentage": percentage,
            })
            habs_true[true_hab] -= percentage_correct
            pred_percentage -= percentage_correct
            if pred_percentage == 0:
                break
            if habs_true[true_hab] == 0:
                continue

        if pred_percentage > 1e-10:
            print("Warning: non matching percentages")

    return pd.DataFrame(outputs)


# # TODO, misschien is bereken percentage correct, wel niet zo interessant
# # want htevolgende kan ook
# df.groupby("shape_id").apply(lambda x: x[x["pred_hab"] == x["true_hab"]].percentage.sum())

In [43]:
a = [1, 1, 2]
b = [1, 2, 3]
dict(zip(a, b))

{1: 2, 2: 3}

In [ ]:
gdf_veg2hab["percentage_correct"] = "blabla"
gdf_veg2hab["PercentageCorrect1"] = "blabla"
gdf_veg2hab.to_file...

In [50]:
habs1 = {"H000": 100}
habs2 = {"H000": 100}

# assert berekend_percentage_correct(habs1, habs2) == 100.0
bereken_percentage_confusion_matrix(habs1, habs2)

[{'pred_hab': 'H000', 'true_hab': 'H000', 'percentage': 100}]

In [53]:
habs_pred = {"H3": 50, "H2": 50}
habs_true = {"H1": 60, "H2": 40}

# assert berekend_percentage_correct(habs_pred, habs_true) == 40
bereken_percentage_confusion_matrix(habs_pred, habs_true)
#  == [
#     {"true": "H2", "pred": "H2", "percentage": 40},
#     {"true": "H1", "pred": "H3", "percentage": 50},
#     {"true": "H1", "pred": "H2", "percentage": 10}
# ]

,pred_hab,true_hab,percentage
0,H2,H2,40
1,H3,H1,50
2,H2,H1,10


In [39]:
# hierna misschien omzetten exploden?

df = pd.DataFrame({
    "shape_id": [1, 1, 1],
    "HabtypePred": ["H2", "H3", "H2"],
    "HabtypeTrue": ["H2", "H1", "H1"],
    "Percentage": [40, 50, 10],
    "Oppervlak": [100, 100, 100],
})

In [40]:
df["Effectief_Oppervlak"] = df["Percentage"] / 100 * df["Oppervlak"]
df = df.drop(columns=["shape_id", "Percentage", "Oppervlak"])

In [41]:
tmp = df.groupby(["HabtypePred", "HabtypeTrue"]).sum()
tmp

Effectief_Oppervlak
HabtypePred HabtypeTrue                     
H2          H1                          10.0
            H2                          40.0
H3          H1                          50.0

In [42]:
# TODO, willen we exact dezelfde indices als columns?!
tmp.unstack().fillna(0)

Effectief_Oppervlak      
HabtypeTrue                  H1    H2
HabtypePred                          
H2                         10.0  40.0
H3                         50.0   0.0

In [ ]:

gdf = gpd.GeoDataFrame(data = {
    "Habtype1": ["H123", "H234", "H345", "H456"],
    "Perc1": [80, 20, 40, 40],
    "Habtype2": ["H123", "H234", "H345", "H456"],
    "Perc2": [80, 20, 40, 40],
    "Habtype3": [None, None, None, "H456"],
    "Perc3": [0, 0, 0, 100]
    },
    geometry=gpd.points_from_xy([0, 1, 2, 3], [0, 1, 2, 3])
)

# onze = gpd.read_file("testing/omgezette_vegetatiekarteringen/GR/NM vegetatiekartering RuitenAa2020.shp")[["Habtype1", "Habtype2", "Habtype3", "geometry"]]



# def spatial_join(gdf1, gdf2):
#     overlayed = gpd.overlay(gdf1, gdf2, how="intersection")
#     mask = overlayed.area > 1
#     print(f"Dropping {(~mask).sum()} rows (presumed rounding errors) with a combined area of {overlayed[~mask].area.sum()} m²")
#     return overlayed[overlayed.area > 1]

# spatial_join(onze, hab)
